In [1]:
import os
import json
import numpy as np
import cv2
from PIL import Image

# Define paths
image_dir = 'D:\BTP\Dataset\set_btp\set_1\set_1'
annotation_dir = 'D:\BTP\Dataset\btp_json\json_1'
mask_dir = 'D:\BTP\Dataset\masks\mask_1'
os.makedirs(mask_dir, exist_ok=True)

# Color definitions for regions (BGR)
colors = {
    'heart': (0, 0, 255),    # Red
    'liver': (0, 255, 0),    # Green
    'stomach': (0, 255, 255) # Yellow
}

for image_file in os.listdir(image_dir):
    if not image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    basename = os.path.splitext(image_file)[0]
    json_file = os.path.join(annotation_dir, basename + '.json')
    image_path = os.path.join(image_dir, image_file)

    # Check if corresponding JSON exists
    if not os.path.exists(json_file):
        print(f"Skipping {image_file}: JSON annotation not found.")
        continue

    # Load image dimensions
    image = Image.open(image_path)
    img_width, img_height = image.size

    # Load annotation data
    with open(json_file, 'r') as file:
        data = json.load(file)

    # Determine original annotation dimensions
    if 'imageWidth' in data and 'imageHeight' in data:
        orig_width, orig_height = data['imageWidth'], data['imageHeight']
    else:
        # Fallback method
        all_x = [p[0] for shape in data['shapes'] for p in shape['points']]
        all_y = [p[1] for shape in data['shapes'] for p in shape['points']]
        orig_width, orig_height = int(max(all_x)), int(max(all_y))

    # Scaling factors
    x_scale = img_width / orig_width
    y_scale = img_height / orig_height

    # Create empty color mask
    mask = np.zeros((img_height, img_width, 3), dtype=np.uint8)

    # Draw scaled polygons
    for shape in data['shapes']:
        label = shape['label']
        points = np.array([[int(p[0]*x_scale), int(p[1]*y_scale)] for p in shape['points']], np.int32)
        cv2.fillPoly(mask, [points], colors[label])

    # Save mask image
    mask_path = os.path.join(mask_dir, basename + '_mask.png')
    cv2.imwrite(mask_path, mask)

    print(f"Generated mask for {image_file} at {mask_path}")

print("\nAll masks generated successfully.")


Skipping 104.jpg: JSON annotation not found.
Skipping 109.jpg: JSON annotation not found.
Skipping 11.jpg: JSON annotation not found.
Skipping 117.jpg: JSON annotation not found.
Skipping 118.jpg: JSON annotation not found.
Skipping 126.jpg: JSON annotation not found.
Skipping 132.jpg: JSON annotation not found.
Skipping 135.jpg: JSON annotation not found.
Skipping 139.jpg: JSON annotation not found.
Skipping 140.jpg: JSON annotation not found.
Skipping 141.jpg: JSON annotation not found.
Skipping 146.jpg: JSON annotation not found.
Skipping 148.jpg: JSON annotation not found.
Skipping 155.jpg: JSON annotation not found.
Skipping 158.jpg: JSON annotation not found.
Skipping 161.jpg: JSON annotation not found.
Skipping 17.jpg: JSON annotation not found.
Skipping 180.jpg: JSON annotation not found.
Skipping 184.jpg: JSON annotation not found.
Skipping 185.jpg: JSON annotation not found.
Skipping 186.jpg: JSON annotation not found.
Skipping 187.jpg: JSON annotation not found.
Skipping 190

In [1]:
pip install numpy opencv-python Pillow


Note: you may need to restart the kernel to use updated packages.


In [101]:
import json
import numpy as np
import cv2
from PIL import Image

# File paths
# image_path = 'D:\BTP\Dataset\set_btp\set_1\set_1\ 2.jpg'
image_path = 'D:/BTP/Dataset/set_btp/set_1/set_1/589.jpg'
json_path = 'D:/BTP/Dataset/btp_json/json_1/589.json'
# json_path = 'D:\BTP\Dataset\ btp_json\json_1\ 7.json'
mask_path = 'D:/BTP/Dataset/masks/mask_1/589_mask.png'
# mask_path = 'D:\BTP\Dataset\masks\mask_1\ 2_mask.png'

# Load image and get dimensions
image = Image.open(image_path)
img_width, img_height = image.size

# Load JSON data
with open(json_path, 'r') as file:
    data = json.load(file)

# Extract original annotation dimensions (if provided by annotation tool)
if 'imageWidth' in data and 'imageHeight' in data:
    orig_width, orig_height = data['imageWidth'], data['imageHeight']
else:
    # Fallback: infer dimensions from points
    all_x = [p[0] for shape in data['shapes'] for p in shape['points']]
    all_y = [p[1] for shape in data['shapes'] for p in shape['points']]
    orig_width, orig_height = int(max(all_x)), int(max(all_y))

# Scale coordinates
x_scale = img_width / orig_width
y_scale = img_height / orig_height

# Prepare an empty color mask (colored according to region labels)
color_mask = np.zeros((img_height, img_width, 3), dtype=np.uint8)

# Define colors (BGR format)
colors = {
    'heart': (0, 0, 255),    # Red
    'liver': (0, 255, 0),    # Green
    'stomach': (0, 255, 255) # Yellow
}

# Draw scaled polygons onto mask
for shape in data['shapes']:
    label = shape['label']
    points = np.array([[int(p[0]*x_scale), int(p[1]*y_scale)] for p in shape['points']], dtype=np.int32)
    cv2.fillPoly(color_mask, [points], colors[label])

# Save the generated mask
cv2.imwrite(mask_path, color_mask)

print(f"Colored segmentation mask generated and saved at '{mask_path}'")


Colored segmentation mask generated and saved at 'D:/BTP/Dataset/masks/mask_1/589_mask.png'
